## Статья: 
https://habr.com/ru/articles/799725/
- `Про бустинг в целом`: https://translated.turbopages.org/proxy_u/en-ru.ru.70266c94-681c3592-61f7101e-74722d776562/https/www.geeksforgeeks.org/boosting-in-machine-learning-boosting-and-adaboost/

- `Продолжение про бустинг`: https://translated.turbopages.org/proxy_u/en-ru.ru.70266c94-681c3592-61f7101e-74722d776562/https/www.geeksforgeeks.org/ml-gradient-boosting/

## Ютуб:
https://youtu.be/-cp0aVkoI5U?si=I010UW3YTxXhmdIx

## Fridman_MSE
https://machinelearning-basics.com/decision-trees-splitting-criteria-for-classification-and-regression/

- это улучшанная версия MSE в Градиентном бустинге.

`diff = yL - yR`

`improvement = nL * nR * diff^(2) / nL  + nR` - улучшение качества разбиения

- yL : среднее значение с левой стороны узла
- yR : среднее значение в правой части узла .
- nL: количество наблюдений в левой части узла.
- nR: количество наблюдений в правой части узла

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor


class GBCustomRegressor:
    """
    ОСНОВНАЯ ИДЕЯ: обучить модельки предсказывать антиградиент, чтобы лучше минимизировать ошибку

    Условие:
        - Для задачи регрессии лосс это квадратичная ошибка
        - Нужно поддержать все параметры перечисленные в __init__
        - Нужно написать реализацию всех перечисленных в теле класса методов
        - В качестве реализации дерева решения нужно использовать DecisionTreeRegressor.
        - Другими классами из sklearn пользоваться запрещается
    """
    
    def __init__(
            self,
            *,
            learning_rate=0.1,
            n_estimators=100,
            criterion="friedman_mse",
            min_samples_split=2,
            min_samples_leaf=1,
            max_depth=3,
            random_state=None
    ):

        """
        Args:
            learning_rate (float): Скорость обучения или подругому ШАГ, как в градиентном спуске. Defaults to 0.1.
            n_estimators (int): Количество базовых алгоритмов. Defaults to 100.
            criterion (str): отвечает за критерий сплита в дереве. Defaults to "friedman_mse".
            min_samples_split (int):  Узел не должен делиться, если в нем объем выборке равен или меньше чем min samples split.  Defaults to 2.
            min_samples_leaf (int): Именно в листьях должно быть не меньше чем min_sample_leaf выборок. Defaults to 1.
            max_depth (int): Максимальная глубина дерева. Defaults to 3.
            random_state (int): Начальная точка для рандомных чисел. Defaults to None.
        """

        self.learning_rate = learning_rate # использовал
        self.n_estimators = n_estimators # использовал
        self.criterion = criterion # использовал
        self.min_samples_split = min_samples_split # использовал
        self.min_samples_leaf = min_samples_leaf # использовал
        self.max_depth = max_depth # использовал
        self.random_state = random_state # использовал\
        self.trees = []

    @staticmethod
    def MSE_gradient(y_true, y_pred):
        """
        (-1) * grad(1/2(Y - alpha)^ 2 = y - alpha 
        """ 
        return y_true - y_pred

    def fit(self, x, y):
        
        """
        Алгорит обучения: 
        
        1) первоначальному прогнозу присваивается среднее значение y для всех образцов;

        2) рассчитываются остатки модели на основе антиградиента функции потерь;

        3) регрессионное дерево обучается на X и остатках, далее делается прогноз на X;

        4) полученный прогноз добавляется к первоначальному и шаги 2-4 повторяются для каждого дерева;

        5) после обучения всех моделей снова создаётся первоначальный прогноз из шага 1;

        6) далее делаются прогнозы для X_test на обученных деревьях и добавляются к первоначальному;

        7) полученная сумма и будет конечным прогнозом.

        """
        self.base = np.mean(y)
        self.y_pred = np.mean(y)
        # У меня возник вопрос: почему мы инициализируем средним, в итоге понял из-за того что на начальном этапе - это самая оптимальная точка для минимизации фукции MSE, т.к. если взять производную от  MSE и приравнять ее к 0. Мы получим ymax = mean(y) <- вторую производную брать смысла нет, чтобы доказывать что это max...

        for _ in range(self.n_estimators):
            residuals = self.MSE_gradient(y, self.y_pred) # это остатки на которых должна обучиться следующая модель
            # был вопрос: почему мы тут берем антиградиент? Ответ: потому, что наши модельки пытаются предсказать - этот антиградиент, чтобы минимизировать ошибку 
            
            temp_tree = DecisionTreeRegressor(criterion= self.criterion,
                                              min_samples_split= self.min_samples_split,
                                              min_samples_leaf= self.min_samples_leaf,
                                              max_depth= self.max_depth,
                                              random_state= self.random_state)
            
            temp_tree.fit(x, residuals)
            
            self.trees.append(temp_tree)

            self.y_pred += self.learning_rate * temp_tree.predict(x) # вот здесь модель выдает градиент

    
    def predict(self, x):
        summ = 0
        for tree in self.trees:
            tree:DecisionTreeRegressor
            summ += tree.predict(x)
        summ *= self.learning_rate

        return self.base + summ

    @property
    def estimators_(self):
        return self.trees 



In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

class GBCustomClassifier:
    """
    ОСНОВНЫЕ ОТЛИЧИЯ от регрессионной модели:
        - нужно преобразовывать y: onehotencoding (если много классов) / приводить к вероятностному виду используя сигмоиду
        - использование logloss

    Условие:
        - Для бинарной классификации использовать logloss.
        - Нужно поддержать все параметры перечисленные в __init__
        - Нужно написать реализацию всех перечисленных в теле класса методов
        - В качестве реализации дерева решения нужно использовать DecisionTreeRegressor.
        - Другими классами из sklearn пользоваться запрещается
    """
    def __init__(
            self,
            *,
            learning_rate=0.1, 
            n_estimators=100,
            criterion="friedman_mse",
            min_samples_split=2,
            min_samples_leaf=1,
            max_depth=3,
            random_state=None
    ):
        """
        Args:
            learning_rate (float): Скорость обучения или подругому ШАГ, как в градиентном спуске. Defaults to 0.1.
            n_estimators (int): Количество базовых алгоритмов. Defaults to 100.
            criterion (str): отвечает за критерий сплита в дереве. Defaults to "friedman_mse".
            min_samples_split (int):  Узел не должен делиться, если в нем объем выборке равен или меньше чем min samples split.  Defaults to 2.
            min_samples_leaf (int): Именно в листьях должно быть не меньше чем min_sample_leaf выборок. Defaults to 1.
            max_depth (int): Максимальная глубина дерева. Defaults to 3.
            random_state (int): Начальная точка для рандомных чисел. Defaults to None.
        """
        self.learning_rate = learning_rate # использовал
        self.n_estimators = n_estimators # использовал
        self.criterion = criterion # использовал
        self.min_samples_split = min_samples_split # использовал
        self.min_samples_leaf = min_samples_leaf # использовал
        self.max_depth = max_depth # использовал
        self.random_state = random_state # использовал\
        self.trees = []

    @staticmethod
    def sigmoid(y:np.array):
        """
        Формула: 1 / 1 + exp(-x)
        """
        return 1 / (1 + np.exp(-y))

    def fit(self, x, y):

        y_mean = np.mean(y)
        self.base = np.log(y_mean / (1 - y_mean)) if y_mean not in (0, 1) else 0 # изменение от регрессии
        y_pred = np.full_like(y, self.base, dtype= float)
        
        for _ in range(self.n_estimators):
            
            p = self.sigmoid(y_pred) # изменение от регрессии регрессии
            residuals = y - p
            
            temp_tree = DecisionTreeRegressor(min_samples_split= self.min_samples_split,
                                              min_samples_leaf= self.min_samples_leaf,
                                              max_depth= self. max_depth,
                                              random_state= self.random_state,
                                              criterion= self.criterion)
            
            temp_tree.fit(x, residuals, sample_weight= p * (1 - p))
            
            self.trees.append(temp_tree)
            
            y_pred += self.learning_rate * temp_tree.predict(x)

    def predict_proba(self, x):
        logit = self.base + self.learning_rate * np.sum([tree.predict(x) for tree in self.trees])
        return self.sigmoid(logit)
    
    def predict(self, x):
        return (self.predict_proba(x) >= 0.5).astype(int)

    @property
    def estimators_(self):
        return self.trees


In [8]:
a = np.array([1, 2, 3])
b = np.array([6, 4, 5])
np.vstack((a, b)).T[0]

array([1, 6])